<a href="https://colab.research.google.com/github/Szozan/2d-strategic-plan/blob/main/deliverables/Step_3_Audio_Trimmer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script ingests a WAV audio file from the user's Google Drive and then automatically trims off the starting and ending silence. This is to prevent the recording/transcript from over-coding "Silence" when the class has not actually started yet, or ended before the recording ended.

The WAV file should be saved in the user's Google Drive so that it can be access by Colab notebook. To access the user's Google Drive, mount it as a virtual hard drive in the Colab notebook environment. The first time this is run, the user will be prompted with a popup to authorize the Colab notebook to access the user's Google Drive. Accept this prompt.

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


pydub is a library which allows audio file manipulation

In [2]:
!pip install pydub

Import the various libraries needed in this script

In [36]:
import io  # Ensure io is imported for in-memory file operations
import os
import json
import wave
from pydub import AudioSegment

Create the audio WAV object by referencing the path to the WAV in the user's Google Drive. To find this, open the folder icon in Colab (left hand side), then navigate through the folder "drive". The sample_data folder can be ignored, that comes with Colab by default. You can drag and drop the MP3 file below to replace the file path for AudioSegment.from_mp3('your filepath here')

In [31]:
fname = '/content/drive/My Drive/ISEA_Test_Audio/RawInterview.wav' #file path to wav
audio = AudioSegment.from_file(fname, format="wav")

'/content/drive/My Drive/ISEA_Test_Audio/RawInterview.mp3'

Simple call of the object audio to see if it was loaded into Colab. If it was, it will bring up a simple audio player widget with a play/pause button and duration bar.

In [14]:
audio

One option is the user can manually set the start and end point to slice out of the MP3 file. When testing uploads to OpenAI it will be much cheaper to send a small snippet of the audio file rather than the entire thing.

In [15]:
sampleAudio = audio[5000:10000] #slice uses [start time:end time] in milliseconds. This is 00:05.00 to 00:10.00

Test that the sample sliced audio worked

In [16]:
sampleAudio

This option automatically detects the beginning and end of audio and slices the recording to exclude the silence at the beginning and end. You can play with the value of the threshold level variable "dbThreshhold" to define how much noise counts as silence.

In [28]:
dbThreshhold = -50.0 #default is -50.0 (which is extremely quiet). Higher db values are louder (better for hiss, background noise, etc which should count as silence).

In [29]:
def detect_leading_silence(audio, dbThreshhold, chunk_size=10):
    '''
    sound is a pydub.AudioSegment
    silence_threshold in dB
    chunk_size in ms

    iterate over chunks until you find the first one with sound
    '''
    trim_ms = 0 # ms

    assert chunk_size > 0 # to avoid infinite loop
    while audio[trim_ms:trim_ms+chunk_size].dBFS < dbThreshhold and trim_ms < len(audio):
        trim_ms += chunk_size

    return trim_ms

#sound = AudioSegment.from_file("/path/to/file.wav", format="wav")

start_trim = detect_leading_silence(audio, dbThreshhold)
end_trim = detect_leading_silence(audio.reverse(), dbThreshhold)

duration = len(audio)
trimmed_audio = audio[start_trim:duration-end_trim]

Test that the trimmed audio worked

In [30]:
trimmed_audio

Save the automatically trimmed audio as a WAV file in the same folder as the source audio file

In [45]:
trimmed_audio.export("/content/drive/My Drive/ISEA_Test_Audio/trimmedInterview3.wav", format="wav")
drive.flush_and_unmount() #the saved files are in the Colab VM, but asynchronously get saved to the actual Google Drive. Sometimes they may never get saved there. Flushing and remounting the drive can fix this and actually save the file into Google Drive.

In [46]:
drive.mount('/content/drive') #have to remount the drive after flushing it and unmounting it

Mounted at /content/drive
